In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/src/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, missing_data, correlation_data, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

2023-12-15 15:35:16.597 | DEBUG    | utils:df_from_csv:340 - Load DataFrame: /home/sebaxtian/Workspaces/Tangara/tangara-evaluation/src/data/0_raw/tangaras.csv


Total Tangara Sensors: 22


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.378983,-76.541061
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_2BDE,d29e6de,D29ESP32DE52BDE,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
3,TANGARA_39D6,d29e6de,D29ESP32DE539D6,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
4,TANGARA_3B7E,d29e6de,D29ESP32DE53B7E,3.3982086181640625 -76.52595520019531,3.398209,-76.525955


In [3]:
# Filter by ID_TANGARA_REFERENCE and IDS_TANGARA_TARGETS

# Tangara Sensor Reference
ID_TANGARA_REFERENCE=os.getenv("ID_TANGARA_REFERENCE", None)
# Tangara Sensors Target
IDS_TANGARA_TARGETS=os.getenv("IDS_TANGARA_TARGETS", None)

print(f"Tangara Sensor Reference: {ID_TANGARA_REFERENCE}")
print(f"Tangara Sensors Target: {IDS_TANGARA_TARGETS}")

ids = IDS_TANGARA_TARGETS.split(',') if IDS_TANGARA_TARGETS else []
ids.append(ID_TANGARA_REFERENCE)

df_tangaras = df_tangaras[df_tangaras['ID'].isin(ids)]

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Tangara Sensor Reference: TANGARA_5636
Tangara Sensors Target: TANGARA_39D6,TANGARA_2BDE,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_422A,TANGARA_5636,TANGARA_D282
Total Tangara Sensors: 8


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
2,TANGARA_2BDE,d29e6de,D29ESP32DE52BDE,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
3,TANGARA_39D6,d29e6de,D29ESP32DE539D6,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
4,TANGARA_3B7E,d29e6de,D29ESP32DE53B7E,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
5,TANGARA_3BEA,d29e6de,D29ESP32DE53BEA,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
6,TANGARA_421A,d29e6de,D29ESP32DE5421A,3.3982086181640625 -76.52595520019531,3.398209,-76.525955


In [4]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

2023-12-15 15:35:16.646 | DEBUG    | utils:to_timestamp:99 - datetime_iso8601: 2023-11-26T00:00:00-05:00, Timestamp: 1700974800000
2023-12-15 15:35:16.648 | DEBUG    | utils:to_timestamp:99 - datetime_iso8601: 2023-12-10T23:59:59-05:00, Timestamp: 1702270799000


Since: 2023-11-26T00:00:00-05:00 -> 1700974800000, Until: 2023-12-10T23:59:59-05:00 -> 1702270799000
Group by Time: 30s


In [5]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp', GROUP_BY_TIME)
df_temp_raw.head()
#df_temp_raw.shape

2023-12-15 15:35:16.657 | DEBUG    | utils:query_measure:192 - sql_query: SELECT last("tmp") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE52BDE') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("tmp") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE539D6') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("tmp") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE53B7E') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("tmp") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE53BEA') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("tmp") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE5421A') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("tmp") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE5422A') AND time >= 17009

,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_422A,TANGARA_5636,TANGARA_D282
DATETIME,,,,,,,,
2023-11-26 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-26 00:00:30-05:00,29.61,29.81,30.72,31.92,32.05,NaN,NaN,NaN
2023-11-26 00:01:00-05:00,29.53,29.76,30.65,31.82,32.00,NaN,NaN,NaN
2023-11-26 00:01:30-05:00,29.43,29.69,30.60,31.73,31.91,32.61,30.95,30.44
2023-11-26 00:02:00-05:00,29.38,29.59,30.49,31.63,31.83,32.45,30.83,30.31


## Descriptive Statistics

In [6]:
# Describe Data
df_temp_raw.describe()

,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_422A,TANGARA_5636,TANGARA_D282
count,28045.000000,25549.000000,39888.000000,39358.000000,39186.000000,38837.000000,38993.00000,39058.000000
mean,32.589217,33.662891,33.751814,34.571431,34.828413,35.411493,33.94055,33.344837
std,2.854940,3.182601,3.003628,3.089270,3.016924,2.961182,2.91449,2.995960
min,25.740000,23.660000,27.100000,27.600000,0.000000,28.590000,26.88000,26.250000
25%,30.260000,30.910000,31.260000,32.150000,32.480000,33.010000,31.53000,30.910000
50%,32.220000,33.210000,33.450000,34.210000,34.560000,35.130000,33.62000,33.060000
75%,34.680000,36.010000,35.970000,36.730000,36.940000,37.540000,36.06000,35.490000
max,39.700000,41.820000,41.020000,42.410000,41.660000,42.410000,41.05000,40.340000


## Missing Data

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold_data = float(os.getenv("THRESHOLD_MISSING_DATA", None))
print(f'Threshold Missing Data: {int(threshold_data * 100)}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    to_be_checked, data_percent, missing_data_percent = missing_data(df_temp_sensor, threshold_data)
    if to_be_checked:
        print(f"Tangara Sensor: {id_tangara_sensor}, To be checked: {to_be_checked}, Data: {data_percent}%, Missing Data: {missing_data_percent}%, Threshold: {threshold_data}%")

        # Drop Tangara Sensor from original dataframe
        df_temp_raw = df_temp_raw.drop(id_tangara_sensor, axis=1)

2023-12-15 15:35:25.340 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_2BDE, To be checked: True, Data: 65%, Missing Data: 35%, Threshold: 80%
2023-12-15 15:35:25.347 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_39D6, To be checked: True, Data: 59%, Missing Data: 41%, Threshold: 80%
2023-12-15 15:35:25.354 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_3B7E, To be checked: False, Data: 92%, Missing Data: 8%, Threshold: 80%
2023-12-15 15:35:25.358 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_3BEA, To be checked: False, Data: 91%, Missing Data: 9%, Threshold: 80%
2023-12-15 15:35:25.363 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_421A, To be checked: False, Data: 91%, Missing Data: 9%, Threshold: 80%
2023-12-15 15:35:25.368 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_422A, To be checked: False, Data: 90%, Missing Data: 10%, Threshold: 80%
2023-12-15 15:35:25.371 | DEBUG    | utils:missing_

Threshold Missing Data: 80%
Tangara Sensor: TANGARA_2BDE, To be checked: True, Data: 65%, Missing Data: 35%, Threshold: 0.8%
Tangara Sensor: TANGARA_39D6, To be checked: True, Data: 59%, Missing Data: 41%, Threshold: 0.8%


## Data Correlation

In [8]:
# Data Correlation Threshold
threshold_correlation = float(os.getenv("THRESHOLD_CORRELATION_DATA", None))
print(f'Threshold Correlation Data: {int(threshold_correlation * 100)}%')

# Tangara Sensor Reference
ID_TANGARA_REFERENCE=os.getenv("ID_TANGARA_REFERENCE", None)

# Tangara Sensors Target
IDS_TANGARA_TARGETS=os.getenv("IDS_TANGARA_TARGETS", None)
IDS_TANGARA_TARGETS = IDS_TANGARA_TARGETS.split(',') if IDS_TANGARA_TARGETS else ID_TANGARA_REFERENCE

# print(f"Tangara Sensor Reference: {ID_TANGARA_REFERENCE}, Tangara Sensors Target: {IDS_TANGARA_TARGETS}")

# Reference Tangara Sensor
df_tangara_reference = df_temp_raw[[ID_TANGARA_REFERENCE]]

# For each Tangara Sensor Target
for id_tangara in IDS_TANGARA_TARGETS:
    if id_tangara in df_temp_raw.columns:
        # Target Tangara Sensor
        df_tangara_target = df_temp_raw[[id_tangara]]

        # To be checked
        if not math.isnan(df_tangara_reference.std()):
            has_corr, corr = correlation_data(df_tangara_reference, df_tangara_target, threshold_correlation)
            if not has_corr:
                print(f"Tangara Sensor Reference: {ID_TANGARA_REFERENCE}, Tangara Sensors Target: {id_tangara}, There is correlation: {has_corr}, Correlation: {float("{:.2f}".format(corr))}, Threshold: {threshold_correlation}")

                # Drop Tangara Sensor from original dataframe
                df_temp_raw = df_temp_raw.drop(id_tangara, axis=1)

2023-12-15 15:35:25.424 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_3B7E, There is correlation: True, Correlation: 1.0, Threshold: 0.7
2023-12-15 15:35:25.446 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_3BEA, There is correlation: True, Correlation: 0.99, Threshold: 0.7
2023-12-15 15:35:25.462 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_421A, There is correlation: True, Correlation: 0.98, Threshold: 0.7
2023-12-15 15:35:25.483 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_422A, There is correlation: True, Correlation: 1.0, Threshold: 0.7
2023-12-15 15:35:25.506 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_5636, There is correlation: True, Correl

Threshold Correlation Data: 70%


In [9]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")

2023-12-15 15:35:26.606 | DEBUG    | utils:df_to_csv:311 - Save DataFrame: /home/sebaxtian/Workspaces/Tangara/tangara-evaluation/src/data/0_raw/temp_raw.csv
